In [1]:
import pygrib
import numpy as np

In [13]:
grbs = pygrib.open('../data/GFS/GFS_2021021900+003.grib2')

In [11]:
grbs_ge = pygrib.open('../data/GEFS/GEFS_01_2021022300+480.0p50.grib2')

In [ ]:
grbs_ge.

In [14]:
for grbe in grbs:
    print(grbe)

1:U component of wind:m s**-1 (instant):regular_ll:unknown:level 0:fcst time 3 hrs:from 202102190000
2:V component of wind:m s**-1 (instant):regular_ll:unknown:level 0:fcst time 3 hrs:from 202102190000
3:Temperature:K (instant):regular_ll:isobaricInPa:level 40 Pa:fcst time 3 hrs:from 202102190000
4:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 3 hrs:from 202102190000
5:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 3 hrs:from 202102190000
6:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 3 hrs:from 202102190000
7:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 3 hrs:from 202102190000
8:Temperature:K (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 3 hrs:from 202102190000
9:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 3 hrs:from 202102190000
10:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 20

158:Temperature:K (avg):regular_ll:unknown:level 0:fcst time 0-3 hrs (avg):from 202102190000
159:Temperature:K (avg):regular_ll:unknown:level 0:fcst time 0-3 hrs (avg):from 202102190000
160:Temperature:K (avg):regular_ll:unknown:level 0:fcst time 0-3 hrs (avg):from 202102190000
161:Pressure:Pa (instant):regular_ll:tropopause:level 0:fcst time 3 hrs:from 202102190000
162:Temperature:K (instant):regular_ll:tropopause:level 0:fcst time 3 hrs:from 202102190000
163:U component of wind:m s**-1 (instant):regular_ll:tropopause:level 0:fcst time 3 hrs:from 202102190000
164:V component of wind:m s**-1 (instant):regular_ll:tropopause:level 0:fcst time 3 hrs:from 202102190000
165:Pressure:Pa (instant):regular_ll:maxWind:level 0:fcst time 3 hrs:from 202102190000
166:U component of wind:m s**-1 (instant):regular_ll:maxWind:level 0:fcst time 3 hrs:from 202102190000
167:V component of wind:m s**-1 (instant):regular_ll:maxWind:level 0:fcst time 3 hrs:from 202102190000
168:Temperature:K (instant):regula

In [12]:
for grb in grbs_ge:
    print(grb)

1:Temperature:K (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
2:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
3:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
4:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
5:Temperature:K (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
6:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
7:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
8:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 480 hrs:from 202102230000:pos ens pert 1
9:Te

In [29]:
def grb_to_grid(grb_obj):
    """Takes a single grb object containing multiple
    levels. Assumes same time, pressure levels. Compiles to a cube"""
    n_levels = len(grb_obj)
    levels = np.array([grb_element['level'] for grb_element in grb_obj])
    indexes = np.argsort(levels)[::-1] # highest pressure first
    cube = np.zeros([n_levels, grb_obj[0].values.shape[0], grb_obj[1].values.shape[1]])
    for i in range(n_levels):
        cube[i,:,:] = grb_obj[indexes[i]].values
    cube_dict = {'data' : cube, 'units' : grb_obj[0]['units'],
                 'levels' : levels[indexes]}
    return cube_dict

In [9]:
t2mens = []
for grb in grbs:
    if grb.parameterName == 'Temperature' and grb.level == 2: 
        t2mens.append(grb.values)

In [13]:
t2 = grbs.select(name='2 metre temperature')

In [14]:
t2

[141:2 metre temperature:K (instant):regular_ll:heightAboveGround:level 2 m:fcst time 0 hrs:from 202102190000]

In [21]:
ppn = grbs.select(name='Total Precipitation')

In [30]:
ppn_=grb_to_grid(ppn)

In [34]:
ppn_['data'].shape

(2, 721, 1440)

In [37]:
(ppn_['data'][0] - ppn_['data'][1]).min()

0.0

# SAVE TO RASTER

In [ ]:
#new_dataset = rasterio.open(folder+'res/'+'SUMA_GOES_16_GLM'+doy+'.tif',
new_dataset = rasterio.open(output+nombre+'_correccion.tif',
     'w',
     driver='GTiff',
     height=height,
     width=width,
     count=1,
     dtype='float32',
     crs='+proj=latlong',
     transform=nuevo_affine) 